In [1]:
import numpy as np
import cv2
import pandas as pd
import os
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense

2024-07-27 14:25:01.092266: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 14:25:01.092391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 14:25:01.236358: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
original_path = '/kaggle/input/braintumordataset/brainTumor'
no_tumor_images = os.listdir(original_path+'/no')
yes_tumor_images = os.listdir(original_path+'/yes')
print(len(no_tumor_images), len(yes_tumor_images) )

1500 1500


In [3]:
dataset = []
label = []


In [4]:
for i in range(len(no_tumor_images)):
    no_image = cv2.imread(original_path+'/no/'+no_tumor_images[i])
    yes_image = cv2.imread(original_path+'/yes/'+yes_tumor_images[i])
    no_image = Image.fromarray(no_image, 'RGB')
    yes_image = Image.fromarray(yes_image, 'RGB')
    no_image = no_image.resize((128,128))
    yes_image = yes_image.resize((128,128))
    dataset.append(no_image)
    label.append(0)
    dataset.append(yes_image)
    label.append(1)

In [5]:
dataset = np.array(dataset)
label = np.array(label)
dataset = normalize(dataset, axis=1)

In [6]:
xtrain, xtest, ytrain, ytest = train_test_split(dataset, label, test_size=0.2, random_state = 0)

In [7]:
x_val = xtest[:300]
y_val = ytest[:300]
xtest = xtest[300:600]
ytest = ytest[300:600]

In [8]:
print(x_val.shape, y_val.shape, xtest.shape, ytest.shape)

(300, 128, 128, 3) (300,) (300, 128, 128, 3) (300,)


In [9]:
print(xtrain.shape)

(2400, 128, 128, 3)


# *Building the 1st Architecture*

In [10]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,699,009 (6.48 MB)

 Trainable params: 1,699,009 (6.48 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.fit(xtrain, ytrain, batch_size=32, verbose=1,
          epochs=15, validation_data=(x_val, y_val), shuffle=False)

Epoch 1/15
13/75 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5898 - loss: 0.6755

I0000 00:00:1722090350.654198      74 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1722090350.676158      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


73/75 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6937 - loss: 0.5892

W0000 00:00:1722090352.399131      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.6958 - loss: 0.5870 - val_accuracy: 0.8100 - val_loss: 0.4385
Epoch 2/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8122 - loss: 0.4287 - val_accuracy: 0.8800 - val_loss: 0.2731
Epoch 3/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8536 - loss: 0.3216 - val_accuracy: 0.9133 - val_loss: 0.2281
Epoch 4/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9167 - loss: 0.2183 - val_accuracy: 0.9467 - val_loss: 0.1826
Epoch 5/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9427 - loss: 0.1496 - val_accuracy: 0.9633 - val_loss: 0.1362
Epoch 6/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9760 - loss: 0.0831 - val_accuracy: 0.9633 - val_loss: 0.1254
Epoch 7/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9859 - loss: 0.0473 - val_accuracy: 0.9667 - val_loss: 0.1222
Epoch 8/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9895 - loss: 0.0382 - val_accuracy: 0.9700 - val_loss: 0

In [13]:
loss, accuracy = model.evaluate(xtest, ytest, verbose=0)
print(f"test_loss={loss}, test_accuracy={accuracy}")

test_loss=0.1429937481880188, test_accuracy=0.9700000286102295


In [14]:
print(xtest[10].shape)
print(xtest[11].shape)

(128, 128, 3)
(128, 128, 3)


In [15]:
print(ytest)

[0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 1 0 0 1
 1 1 0 1 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 0 0 0
 1 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 1
 0 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 0 0
 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 1 1 0 1 1 0 0
 0 0 1 1 0 0 0 1 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0 0 0 0 0
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 0
 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0 1
 1 0 0 0]


# *Testing the 1st Architecture*

In [16]:
import numpy as np

def predict_image(model, xtest, ytest, index):
    # Select the input image and add a batch dimension
    input_image = np.expand_dims(xtest[index], axis=0)

    # Predict the label using the model
    prediction = np.round(model.predict(input_image))

    # Get the actual label
    actual_label = ytest[index]

    # Print the results
    print(f"Prediction={prediction}, Actual={actual_label}")

    return prediction, actual_label

# Example usage
predicted_label, actual_label = predict_image(model, xtest, ytest, 0)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 682ms/step
Prediction=[[0.]], Actual=0


# *Building the 2nd Architecture*
<h5>Changing the filter size to (5, 5) instead of (3, 3)</h5>

In [17]:
model_2 = Sequential()

model_2.add(Conv2D(32, (5, 5), input_shape=(128, 128, 3)))
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

model_2.add(Conv2D(64, (5, 5)))
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

model_2.add(Conv2D(128, (5, 5)))
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

model_2.add(Flatten())
model_2.add(Dense(64))
model_2.add(Activation('relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(1))
model_2.add(Activation('sigmoid'))

In [18]:
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 124, 124, 32)   │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 58, 58, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 58, 58, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 25, 25, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 25, 25, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │     1,179,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,438,401 (5.49 MB)

 Trainable params: 1,438,401 (5.49 MB)

 Non-trainable params: 0 (0.00 B)

<h3>2nd Arcitecture Training</h3>

In [19]:
model_2.fit(xtrain, ytrain, batch_size=32, verbose=1,
          epochs=15, validation_data=(x_val, y_val), shuffle=False)

Epoch 1/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.6566 - loss: 0.5999 - val_accuracy: 0.8233 - val_loss: 0.4106
Epoch 2/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8094 - loss: 0.4205 - val_accuracy: 0.8567 - val_loss: 0.2896
Epoch 3/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8606 - loss: 0.3198 - val_accuracy: 0.9033 - val_loss: 0.2138
Epoch 4/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8950 - loss: 0.2489 - val_accuracy: 0.9267 - val_loss: 0.1743
Epoch 5/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9360 - loss: 0.1792 - val_accuracy: 0.9600 - val_loss: 0.1231
Epoch 6/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9593 - loss: 0.1103 - val_accuracy: 0.9500 - val_loss: 0.1501
Epoch 7/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9699 - loss: 0.0920 - val_accuracy: 0.9667 - val_loss: 0.1329
Epoch 8/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9869 - loss: 0.0535 - val_accuracy: 0.9800 - 

<h3>2nd Architecture Evaluation</h3>

In [20]:
loss_2, accuracy_2 = model_2.evaluate(xtest, ytest, verbose=0)
print(f"test_loss={loss_2}, test_accuracy={accuracy_2}")

test_loss=0.1496717780828476, test_accuracy=0.9766666889190674


<h3>2nd Architecture Testing</h3>

In [21]:
predicted_label_2, actual_label_2 = predict_image(model_2, xtest, ytest, 0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 701ms/step
Prediction=[[0.]], Actual=0


# *Building the 3rd Architecture*
<h5>Removing the third convolutional layer</h5>

In [22]:
model_3 = Sequential()

model_3.add(Conv2D(32, (5, 5), input_shape=(128, 128, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Conv2D(64, (5, 5)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Flatten())
model_3.add(Dense(64))
model_3.add(Activation('relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(1))
model_3.add(Activation('sigmoid'))

In [23]:
model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_3.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 124, 124, 32)   │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 58, 58, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 58, 58, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 53824)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │     3,444,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,498,561 (13.35 MB)

 Trainable params: 3,498,561 (13.35 MB)

 Non-trainable params: 0 (0.00 B)

<h3>3rd Architecture Training</h3>

In [24]:
model_3.fit(xtrain, ytrain, batch_size=32, verbose=1,
          epochs=15, validation_data=(x_val, y_val), shuffle=False)

Epoch 1/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.7038 - loss: 0.5621 - val_accuracy: 0.8867 - val_loss: 0.3277
Epoch 2/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8579 - loss: 0.3470 - val_accuracy: 0.9433 - val_loss: 0.1975
Epoch 3/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9317 - loss: 0.1977 - val_accuracy: 0.9200 - val_loss: 0.2241
Epoch 4/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9472 - loss: 0.1427 - val_accuracy: 0.9633 - val_loss: 0.1631
Epoch 5/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9714 - loss: 0.0798 - val_accuracy: 0.9700 - val_loss: 0.1031
Epoch 6/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9821 - loss: 0.0499 - val_accuracy: 0.9700 - val_loss: 0.1038
Epoch 7/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9824 - loss: 0.0412 - val_accuracy: 0.9833 - val_loss: 0.0809
Epoch 8/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9916 - loss: 0.0248 - val_accuracy: 0.9833 - v

<h3>3rd Architecture Evaluation</h3>

In [25]:
loss_3, accuracy_3 = model_3.evaluate(xtest, ytest, verbose=0)
print(f"test_loss={loss_3}, test_accuracy={accuracy_3}")

test_loss=0.19927094876766205, test_accuracy=0.9666666388511658


<h3>3rd Architecture Testing</h3>

In [26]:
predicted_label_3, actual_label_3 = predict_image(model_3, xtest, ytest, 0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
Prediction=[[0.]], Actual=0


# *Building the 4th Architecture*
* <h3>Increased the number of neurons in the fully connected layer from 64 to 128.</h3>
* <h3>Reduced the dropout rate from 0.5 to 0.25.</h3>

In [27]:
model_4 = Sequential()

model_4.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3)))
model_4.add(Activation('relu'))
model_4.add(MaxPooling2D(pool_size=(2, 2)))

model_4.add(Conv2D(64, (3, 3)))
model_4.add(Activation('relu'))
model_4.add(MaxPooling2D(pool_size=(2, 2)))

model_4.add(Conv2D(128, (3, 3)))
model_4.add(Activation('relu'))
model_4.add(MaxPooling2D(pool_size=(2, 2)))

model_4.add(Flatten())
model_4.add(Dense(128))
model_4.add(Activation('relu'))
model_4.add(Dropout(0.25))
model_4.add(Dense(1))
model_4.add(Activation('sigmoid'))

In [28]:
model_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_4.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_14 (Activation)      │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_15 (Activation)      │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_16 (Activation)      │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_17 (Activation)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_18 (Activation)      │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

<h3>4th Architecture Training</h3>

In [29]:
model_4.fit(xtrain, ytrain, batch_size=32, verbose=1,
          epochs=15, validation_data=(x_val, y_val), shuffle=False)

Epoch 1/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.7015 - loss: 0.5674 - val_accuracy: 0.8200 - val_loss: 0.3904
Epoch 2/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8412 - loss: 0.3447 - val_accuracy: 0.9200 - val_loss: 0.1981
Epoch 3/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9180 - loss: 0.1961 - val_accuracy: 0.9533 - val_loss: 0.1729
Epoch 4/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9727 - loss: 0.0925 - val_accuracy: 0.9633 - val_loss: 0.1423
Epoch 5/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9888 - loss: 0.0433 - val_accuracy: 0.9733 - val_loss: 0.1295
Epoch 6/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9933 - loss: 0.0225 - val_accuracy: 0.9733 - val_loss: 0.1246
Epoch 7/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9941 - loss: 0.0202 - val_accuracy: 0.9600 - val_loss: 0.1525
Epoch 8/15
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9931 - loss: 0.0238 - val_accuracy: 0.9733 - v

<h3>4th Architecture Evaluation</h3>

In [30]:
loss_4, accuracy_4 = model_4.evaluate(xtest, ytest, verbose=0)
print(f"test_loss={loss_4}, test_accuracy={accuracy_4}")

test_loss=0.1207980364561081, test_accuracy=0.9633333086967468


<h3>4th Architecture Testing</h3>

In [31]:
predicted_label_4, actual_label_4 = predict_image(model_4, xtest, ytest, 0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
Prediction=[[0.]], Actual=0
